# Elasticsearch To OpenAI

In [1]:
import json
import requests
import tiktoken
import openai
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run Elasticsearch

In [2]:
# Get an Elasticsearch client
es_client =  Elasticsearch('http://localhost:9200')

In [3]:
# Verify Connection to Elasticsearch
es_client.info()

ObjectApiResponse({'name': '2cb954d905de', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'LDOx-9RnQU2nl-wYcko81A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Get the Data

In [4]:
# Retrieve the data
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

# Parse the documents
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Index the Data

In [5]:
# Create an index in Elasticsearch
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"}
        }
    }
}

index_name = "user_questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'user_questions'})

In [6]:
# Fit an Elasticsearch client index with the documents
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 40.25it/s]


## Search the data

In [7]:
query = 'How do I execute a command in a running docker container?'

In [8]:
# Query the document for the query and retrieve a response
def elastic_search(query):
    # Create a list that holds the results
    result_docs = []
    
    # Query format for the data
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    # Retrieve the responses
    response = es_client.search(index=index_name, body=search_query)

    # Retrieve the results
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs

In [9]:
elastic_search(query)

[83.526276, 75.62604, 72.16631, 50.787666, 49.62014]

## Filter the data

In [10]:
# Create a filtered index
course_index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

course_index_name = "course_questions"

es_client.indices.create(index=course_index_name, body=course_index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [11]:
# Fit an Elasticsearch client index with the documents
for doc in tqdm(documents):
    es_client.index(index=course_index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 40.89it/s]


In [12]:
# Query the filtered document for the query and retrieve a response
def elastic_filtered_search(query):
    # Create a list that holds the results
    filtered_result_docs = []
    
    # Query format for the data
    filtered_search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    # Retrieve the responses
    filtered_response = es_client.search(index=course_index_name, body=filtered_search_query)

    # Retrieve the results
    for hit in filtered_response['hits']['hits']:
        filtered_result_docs.append(hit['_source'])

    return filtered_result_docs

In [13]:
elastic_filtered_search(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Build a Prompt

In [14]:
# Build a prompt
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION base on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"    

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
# Search the document
search_results = elastic_filtered_search(query)

# Use the search response to build the context
prompt = build_prompt(query, search_results)

In [16]:
len(prompt)

1462

In [17]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION base on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT: 
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

## Number of Tokens in the Query

In [18]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [19]:
len(encoding.encode(prompt))

323

## Bonus:  Send the prompt to OpenAI

In [20]:
# Get an OpenAI client
client = OpenAI()

In [21]:
# Send prompt to OpenAI
def llm(prompt):
    response = client.chat.completions.create(model='gpt-4o', messages=[{"role": "user", "content": prompt}])
    return response.choices[0].message.content

In [22]:
# Perform a search, generate a prompt and send a prompt to OpenAI
def elastic_rag(query):
    search_results = elastic_filtered_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    
    return answer

In [23]:
print(elastic_rag(query))

To execute a command in a running Docker container, follow these steps:

1. First, identify the container ID of the running container by using the `docker ps` command:
   ```
   docker ps
   ```

2. Once you have the container ID, use the `docker exec` command to run a command inside the container. For example, to start a bash session, use:
   ```
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of the container you want to access.
